In [1]:
# set up the slide show
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
    'theme': 'solarized',
    'width': 1024,
    'height': 768,
    'transition': 'none',
    'start_slideshow_at': 'selected',
})

{'height': 768,
 'start_slideshow_at': 'selected',
 'theme': 'solarized',
 'transition': 'none',
 'width': 1024}

In [2]:
# CSS overrides
from IPython.core.display import HTML
css = open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

# A Relational<br/>Espresso

##### A Sip of the Essence of SQL in Python

## This Talk

Understand *why* you would use a 

**relational database** 

in your application

## This Talk

Learn how to **connect** Python to an RDBMS

## This Talk

Obtain a **solid foundation** for *learning more*

There are many, many 

*storage paradigms* and technologies

## Why relational databases?

In particular, what accounts for relational databases’

*enduring popularity?*

An *annecdote* from the **distant past**…


(ca. 1995)

## Why relational databases?

> If you believed everything in the **object database** vendors’ literature, then you’d be surprised that Larry Ellison still has $100 bills to *fling to peasants* as he roars past in his Acura NSX. 
>
> — Philip Greenspun

## Why relational databases?

> The relational database management system should have been **crushed** long ago under the weight of this *superior technology*, introduced with tremendous hype in the mid-1980s.
>
> — Philip Greenspun

Anyone ever used an object database?

(not an ORM; a true ODBMS…)

## So… Why relational databases?

* Declarative querying
* ACID
* Performance

## Declarative querying

Specify *what* you want, not **how** to get it.

## What we want

> Retrieve the value(s) of the `group_name` field
> from the `user_group` table
> but only for people whose name is `Daniel Rocco`

## What we want

    SELECT group_name
    FROM user_group
    WHERE name = 'Daniel Rocco';

## How?

## Hmmm… don't know don't care

![title](dont_care.jpg)

Why is declarative querying interesting and important?

Make simple things *easy*, make difficult things **possible**

SQL is a *quintessential* example of this principle

The query language provides a *clean abstraction* between the **application program** and the **data**

Not only is working with the data simplified, an RDBMS *isolates* your data from the harsh environment of an **application program**

> With an ODBMS, the application program is **directly writing slots** in objects stored in the database. A bug in the application program may translate *directly into corruption of the database*, one of an organization’s **most valuable assets**.
>
> — Philip Greenspun

Equally true for **many other** storage systems, 

e.g. flat files, pickle, etc.

## ACID

Transactions should be

**Atomic**, **Consistent**, **Independent**, and **Durable**

## Atomic

A transaction either happens or it doesn't

![IBM](ibm.jpg)

## Atomic

If, after 1,000 statements, the 1,001<sup>st</sup> **blows up**, the database should behave as if *none* of the statements ever happened

## Consistent

A transaction moves the database 

*from one consistent state to another*

## Consistent

(although *consistent* doesn't necessarily mean<br/>
**expected** or **desirable**)

## Independent

*Concurrent transactions* operate as if they **executed alone**

## Durable

Once a transaction completes, its *effect on the database* should be **permanent**

## These are *Hard* Guarantees to Offer

(especially when you consider…)

## Performance

Disk storage, indexes, query planner and optimizer, statistics, table spaces, …

## Why relational databases?

* Declarative querying: power & ease of use
* ACID: rigorous guarantees even with concurrent access
* Performance

## SQL & Python

DBAPI, SQLAlchemy, sqlite3, tons of third-party drivers, etc.

*Great, dan! But how do I get started?*

## Mmm… batteries…

SQLite binding built into the standard library

In [3]:
import sqlite3

In [4]:
import os

try:
    os.unlink('hello.db')
except FileNotFoundError:
    pass

## Making Connections

Connect to an SQLite database,<br/>
creating it first if it doesn't already exist

In [5]:
db = sqlite3.connect('hello.db')

## Look! It's a File!

In [6]:
!stat -c '%sB %n' hello.db

0B hello.db


## Python ↔ SQL

That `db` variable holds a connection to the database

## Python ↔ SQL

We *execute* SQL commands by passing them to the appropriate `db` methods.

## Python ↔ SQL

The result of a command is a *cursor* object, a reference to the **result** of our statement.

Example: Get all rows & columns from the `user` table

SQL:

    SELECT *
    FROM user;


(‘`*`’ means “all fields/columns”)

In Python, wrap the SQL in a multiline string:

    '''
        SELECT *
        FROM user;
    '''

Run a command by passing it to the `execute` method:

In [8]:
cursor = db.execute('''
    SELECT *
    FROM user;
''')

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: no such table: user

Well, **that** didn’t work…

I suppose we’ll need to create some tables!

## Tables for Your Data


CREATE TABLE, unsurprisingly, creates a new table!<br/>
This one is named `hello`

In [9]:
cursor = db.execute('''
    CREATE TABLE hello (
        data text
    );
''')

Inside the parens are the **fields** (columns) in the table, each of which has a
**name** (data) and a **type** (text)

## Put Information In…

In [10]:
cursor = db.execute('''
    INSERT INTO hello
    VALUES ('world!');
''')

## … and Get It Back Out

In [11]:
cursor = db.execute('''
    SELECT *
    FROM hello;
''')

for row in cursor:
    print(row)

('world!',)


## Concurrency, anyone?

In [12]:
conn2 = sqlite3.connect('hello.db')

cursor2 = conn2.execute('''
    SELECT data
    FROM hello;
''')

for row in cursor2:
    print(row)

              ^^^^
(uhm… something’s missing…)

## WAT?!

Dude… where's my data?

## Transactional Detour

A *transaction* is an **atomic group of statements** 

## Transactional Detour

Because of **Consistency/Independence**, data from an in-flight transaction is *invisible* within a second transaction

## Transactional Detour

`COMMIT`: save and close the transaction

`ROLLBACK`: revert to the state before the transaction

## Transactional Detour

So, if we commit transaction 1:

In [13]:
db.commit()

## Transactional Detour

its data will be *persisted* and therefore <br/>
**visible** to transaction 2:

In [14]:
cursor2 = conn2.execute('''
    SELECT *
    FROM hello;
''')

for row in cursor2:
    print(row)

('world!',)


In [15]:
conn2.close()

## Synthetic Identifiers

It is often convenient to allow the RDBMS to assign a<br/>*unique identitfier* to each row:

In [16]:
cursor = db.execute('''
    CREATE TABLE user (
        id INTEGER PRIMARY KEY,
        user_name VARCHAR UNIQUE,
        first_name VARCHAR,
        last_name VARCHAR
    );
''')

SQL dialects differ; `INTEGER PRIMARY KEY` is SQLite magic for *automatically increasing integer*

We can now omit `id` when *inserting rows*

In [17]:
cursor = db.execute('''
  INSERT INTO user 
    (user_name, first_name, last_name) 
  VALUES
    ('d@rocco.com', 'Daniel', 'Rocco'),
    ('a3@rocco.com', 'Ariadne', 'Rocco');
''')

and the RDBMS will *supply the missing values*

In [18]:
rows = db.execute('''
    SELECT id, user_name, first_name
    FROM user;
''')

print(*rows, sep='\n')

(1, 'd@rocco.com', 'Daniel')
(2, 'a3@rocco.com', 'Ariadne')


## Change Things Up

`ALTER TABLE` statements allow us to 

*add new columns* 

to an existing table, among other operations.

In [19]:
cursor = db.execute('''
    ALTER TABLE user
    ADD COLUMN password VARCHAR(60)
''')

`UPDATE` changes *certain values* in a table’s rows, 

subject to a filter

Here’s an `UPDATE` to store a user’s password. Since we’re *awesome*, we’ll use **cutting-edge hashed passwords**!

In [20]:
import bcrypt

# Securely hash a cleartext pw
password = b'asdf'
hashed = bcrypt.hashpw(
    password, bcrypt.gensalt())

hashed

b'$2b$12$8m7j.WT8XxYx/YGhuI1at.6YlEw9eUf9FdV4yKWki1SMwrMJ91hSG'

So our variable `hashed` contains the hashed password,<br/>
which we then add to the user’s record

In [21]:
cursor = db.execute('''
    UPDATE user
    SET password = ?
    WHERE user_name = 'd@rocco.com'
''', [hashed])

The ‘`?`’ is a *query parameter*,<br/>
a bridge between a *program variable*<br/>
and an **SQL expression**

    ··.·······('''
        UPDATE user
        SET password = ?
        WHERE user_name = '···········'
    ''', [hashed])

For each ‘`?`’, we need to provide a value to <br/>*“fill in the blank”*

    ··.·······('''
        UPDATE ···
        SET password = ? ← `hashed` goes here
        WHERE ···
    ''', [hashed])

Query parameters provide *safety*, protecting our program from malicious user data…

https://xkcd.com/327/

Last example: a *high-level* Python function

backed by the **data store** 

*How do we check a user_name/password combo for validity?*

Here’s a query to retrieve the stored password 

In [22]:
user_name = 'd@rocco.com'

rows = db.execute('''
    SELECT password
    FROM user
    WHERE user_name = ?;
''', [user_name])

print(*rows, sep='\n')

(b'$2b$12$8m7j.WT8XxYx/YGhuI1at.6YlEw9eUf9FdV4yKWki1SMwrMJ91hSG',)


But imagine if we put code like this everywhere…

A key *design challenge* when building 

*DB-driven applications* 

is to **maintain encapsulation**

**Separation of Concerns**

Keep *persistence* separate from *application logic*

**Separation of Concerns**

View *persistence* like an internal *API*

We’ll use a *custom exception class* to indicate **missing users**

*without reference to how users are stored!*

In [23]:
class NoSuchUser(Exception): 
    pass

Our user storage API needs a mechanism 

to get a user’s saved password hash

Given a *user name*,

return the existing user’s **password hash**,

or **raise** `NoSuchUser` if the user name doesn’t exist

In [24]:
def get_password(user_name):
    row = db.execute('''
        SELECT password
        FROM user
        where user_name = ?;
    ''', [user_name]).fetchone()

    try:
        return row[0]
    except TypeError:
        raise NoSuchUser(user_name) \
            from None

In [25]:
get_password('d@rocco.com')

b'$2b$12$8m7j.WT8XxYx/YGhuI1at.6YlEw9eUf9FdV4yKWki1SMwrMJ91hSG'

In [26]:
get_password('bo@gus.com')

NoSuchUser: bo@gus.com

Finally, `check_password`:<br/>
return `True` if the user exists **and** the supplied password hashes correctly, `False` *if either condition fails*

In [27]:
def check_password(user_name, password):
    try:
        hashed = get_password(user_name)
        return hashed == \
            bcrypt.hashpw(password, 
                          hashed) 
    except NoSuchUser:
        return False

In [28]:
check_password('d@rocco.com', b'12345')

False

In [29]:
check_password('d@rocco.com', b'asdf')

True

In [30]:
check_password('bo@gus.com', b'asdf')

False

In [31]:
# db.close()

You now know how to connect a Python application to an RDBMS, and why you might want to do so

The **absolute next thing** you *must* do is learn about `JOIN`

Never stop learning: http://philip.greenspun.com/sql/
        
or just search for *“sql for web nerds”*

♥,

@drocco007